LS-Nikko동제련 박용선

# 문제 1

타이타닉의 train 데이터에서 1) 전체 생존률과 2) 생존자의 총 인원수, 사망자의 총 인원수를 구하세요.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_df = pd.read_csv("data/titanic/train.csv", index_col='PassengerId')
print(train_df.shape)
train_df.head()

(891, 11)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train_df.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
# 1) 전체 생존률 
print("전체생존률 :", round(train_df['Survived'].mean(),2))

전체생존률 : 0.38


In [5]:
# 2) 생존자의 총 인원수, 사망자의 총 인원수
mask = train_df['Survived'] == 1
print("생존자 총 인원수:", train_df.loc[mask, ['Survived']].count())

mask1 = train_df['Survived'] == 0
print("사망자 총 인원수:", train_df.loc[mask1, ['Survived']].count())

# or 0 = 사망자, 1 = 생존자
train_df['Survived'].value_counts()

생존자 총 인원수: Survived    342
dtype: int64
사망자 총 인원수: Survived    549
dtype: int64


0    549
1    342
Name: Survived, dtype: int64

# 문제 2

`Survived` 컬럼에는 0/1 값이 있습니다. 이는 직관적이지 않기에, 이를 표현하는 별도 컬럼 `Survived-Label`을 만들어봅시다.

+ `0` : `사망`
+ `1` : `생존`

In [6]:
survived_mapping = {0:'사망', 1:'생존'}
train_df['Survived-Lable'] = train_df['Survived'].map(survived_mapping)
train_df[['Survived', 'Survived-Lable']].head()

,Survived,Survived-Lable
PassengerId,,
1,0,사망
2,1,생존
3,1,생존
4,1,생존
5,0,사망


# 문제 3

`문제 2`와 마찬가지로, `Pclass` 컬럼 역시 `Pclass-Label` 컬럼을 만들어봅시다.

+ `1` : `퍼스트 클래스`
+ `2` : `비즈니스`
+ `3` : `이코노미`

In [7]:
Pclass_mapping = {1:'퍼스트 클래스', 2:'비즈니스', 3:'이코노미'}
train_df['Pclass-Lable'] = train_df['Pclass'].map(Pclass_mapping)
train_df[['Pclass', 'Pclass-Lable']].head()

,Pclass,Pclass-Lable
PassengerId,,
1,3,이코노미
2,1,퍼스트 클래스
3,3,이코노미
4,1,퍼스트 클래스
5,3,이코노미


# 문제 4

`Pclass`별 생존율을 출력해봅시다.

Hint) `pivot_table`이나 `group_by`를 활용해볼 수 있습니다.

In [8]:
train_df.pivot_table(index='Pclass', values='Survived', aggfunc='mean')

,Survived
Pclass,
1,0.629630
2,0.472826
3,0.242363


# 문제 5

`Embarked` 컬럼도 마찬가지로 `Embarked-Label` 컬럼을 만들어봅시다.

- `C` : `Cherbourg`
- `S` : `Southampton`
- `Q` : `Queenstown`

In [9]:
Embarked_mapping = {'C':'Cherbourg', 'S':'Southampton', 'Q':'Queenstown'}
train_df['Embarked-Label'] = train_df['Embarked'].map(Embarked_mapping)
train_df[['Embarked', 'Embarked-Label']].head()

,Embarked,Embarked-Label
PassengerId,,
1,S,Southampton
2,C,Cherbourg
3,S,Southampton
4,S,Southampton
5,S,Southampton


# 문제 6

`Embarked-Label`별 생존율을 출력해보세요.

In [10]:
train_df.pivot_table(index='Embarked-Label', values = 'Survived')

,Survived
Embarked-Label,
Cherbourg,0.553571
Queenstown,0.389610
Southampton,0.336957


# 예시) 머신러닝 알고리즘 적용

```python
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier  # Decision Tree
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.svm import SVC

# train/test 데이터 준비
column_names = ['Age', 'Pclass']  # FIXME: Feature 컬럼 지정
train_x = train_df[column_names]
train_y = train_df['Survived']
test_x = test_df[column_names]

# DecisionTreeClassifier를 통해 학습
clf = DecisionTreeClassifier(max_depth=5)
clf.fit(train_x, train_y)

# 필드별 중요도를 차트로 그리기
pd.Series(
    clf.feature_importances_,
    index=train_x.columns).sort_values().plot(kind='barh')

# test_x 데이터로 생존여부 예측
clf.predict(test_x)
```

# 문제 7 : One Hot Encoding

`Pclass` 컬럼에 대해, `Pclass-1`, `Pclass-2`, `Pclass-3` 컬럼을 만들어주세요.

+ `Pclass-1` : `Pclass=1`이면 `True`, 아니면 `False`
+ `Pclass-2` : `Pclass=2`이면 `True`, 아니면 `False`
+ `Pclass-3` : `Pclass=3`이면 `True`, 아니면 `False`

In [11]:
train_df['Pclass-1'] = train_df['Pclass'].map({1:True,2:False,3:False})
train_df['Pclass-2'] = train_df['Pclass'].map({1:False,2:True,3:False})
train_df['Pclass-3'] = train_df['Pclass'].map({1:False,2:False,3:True})
train_df[['Pclass', 'Pclass-1','Pclass-2','Pclass-3']].head(15)

,Pclass,Pclass-1,Pclass-2,Pclass-3
PassengerId,,,,
1,3,False,False,True
2,1,True,False,False
3,3,False,False,True
4,1,True,False,False
5,3,False,False,True
6,3,False,False,True
7,1,True,False,False
8,3,False,False,True
9,3,False,False,True


# 문제 8

`Age` 컬럼에 대해, 다음 정보를 출력해주세요.

+ 평균 (mean)
+ 분산 (standard deviation, std)
+ 가장 나이가 많은 사람 (max)
+ 가장 나이가 적은 사람 (min)

> `.describe()` 를 활용해보실 수도 있습니다.

In [12]:
train_df['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [13]:
print('평균: ',train_df['Age'].mean())
print('분산: ',train_df['Age'].std())
print('가장 나이가 많은 사람: ', train_df.loc[train_df['Age'].max(),'Name'])

평균:  29.69911764705882
분산:  14.526497332334044
가장 나이가 많은 사람:  Dowdell, Miss. Elizabeth


In [14]:
# min = 0.42
print('가장 나이가 적은 사람: ', train_df.loc[train_df['Age']==0.42, 'Name'])

가장 나이가 적은 사람:  PassengerId
804    Thomas, Master. Assad Alexander
Name: Name, dtype: object


# 문제 9

객실 등급(`Pclass`)별 `Age` 평균을 구하세요.

In [15]:
train_df.pivot_table(index='Pclass', values='Age')

,Age
Pclass,
1,38.233441
2,29.877630
3,25.140620


# 문제 10

다음 `Age` 구간 마다의 생존율을 구하세요.

+ 나이가 15세 미만인 승객.
+ 나이가 15세 이상이고 30세 미만인 승객.
+ 나이가 30세 이상인 승객.

In [16]:
mask_under15 = train_df['Age'] < 15
mask_btw15n30 = (train_df['Age'] >= 15) & (train_df['Age'] < 30)
mask_over30 = train_df['Age'] >= 30

print("나이가 15세 미만인 승객의 생존율: ",round(train_df.loc[mask_under15, 'Survived'].mean(),2))
print("나이가 15세 이상이고 30세 미만인 승객의 생존율: ",round(train_df.loc[mask_btw15n30, 'Survived'].mean(),2))
print("나이가 30세 이상인 승객의 생존율: ",round(train_df.loc[mask_over30, 'Survived'].mean(),2))


나이가 15세 미만인 승객의 생존율:  0.58
나이가 15세 이상이고 30세 미만인 승객의 생존율:  0.36
나이가 30세 이상인 승객의 생존율:  0.41


# 문제 11

`Age`가 `NaN`인 승객의 생존율과 아닌 승객의 생존율을 구하세요.

In [17]:
mask_null = train_df['Age'].isna() 
#mask_null = train_df['Age'].isfill()
print("Age가 NaN인 승객의 생존율: ",round(train_df.loc[mask_null, 'Survived'].mean(),2))
print("Age Data가 있는 승객의 생존율: ",1 - round(train_df.loc[mask_null, 'Survived'].mean(),2))


Age가 NaN인 승객의 생존율:  0.29
Age Data가 있는 승객의 생존율:  0.71


# 문제 12

`Age`가 `NaN`인 승객의 `Age`에, 그 승객이 속한 `Pclass`의 `Age` 평균을 대입해주세요.

In [36]:
pclass_age_df = train_df.pivot_table(index='Pclass', values='Age', aggfunc='mean')

for pclass in [1, 2, 3]:
    mask = train_df['Age'].isna() & (train_df['Pclass'] == pclass)
    train_df.loc[mask, 'Age'] = pclass_age_df.loc[pclass, 'Age']

train_df['Age']

,Age
Pclass,
1,38.233441
1,38.233441


# 문제 13

형제/자매/배우자(SibSp)도 없고, 부모/자식(Parch)도 없는 사람을 `싱글`이라 하겠습니다. `싱글` 컬럼을 만들어주시고 `싱글` 여부에 따른 생존율을 구해주세요.

In [19]:
# 싱글: 1, 그외: 0
train_df['싱글'] = np.where((train_df['SibSp']==0) & (train_df['Parch'] == 0), 1, 0)

round(train_df.pivot_table(index='싱글', values='Survived'),2)


,Survived
싱글,
0,0.51
1,0.30


# 문제 14

`가족수` 컬럼을 만들어주세요. `가족수 = SibSp + Parch + 1` 수식을 따릅니다.

그리고, `가족수`별 생존율도 구해주세요.

In [20]:
train_df['가족수'] = train_df['SibSp'] + train_df['Parch'] + 1
round(train_df.pivot_table(index='가족수', values='Survived'),2)

,Survived
가족수,
1,0.30
2,0.55
3,0.58
4,0.72
5,0.20
6,0.14
7,0.33
8,0.00
11,0.00


# 문제 15

`가족수` 컬럼이 너무 세분화되어있습니다. 이를 구역으로 나눠보겠습니다.

+ `싱글` : `가족수<=1`
+ `핵가족` : `1<가족수<5`
+ `대가족` : `가족수>=5`

위를 참조해서 `가족형태` 컬럼을 만들어주시고, `가족형태`별 생존율을 구해주세요.

In [21]:
conditions = [
    (train_df['가족수'] <= 1),
    (train_df['가족수'] > 1) & (train_df['가족수'] < 5),
    (train_df['가족수'] >= 5)]
choices = ['싱글', '핵가족', '대가족']
train_df['가족형태'] = np.select(conditions, choices)

round(train_df.pivot_table(index = '가족형태', values = 'Survived'),2)

,Survived
가족형태,
대가족,0.16
싱글,0.30
핵가족,0.58


# 문제 16 : One Hot Encoding

`가족형태` 컬럼에 대해, `가족형태-싱글`, `가족형태-핵가족`, `가족형태-대가족` 컬럼을 만들어주세요.

+ `가족형태-싱글` : `가족형태=싱글`이면 True, 아니면 False
+ `가족형태-핵가족` : `가족형태=핵가족`이면 True, 아니면 False
+ `가족형태-대가족` : `가족형태=대가족`이면 True, 아니면 False

In [22]:
train_df['가족형태-싱글'] = train_df['가족형태'].map({'싱글':True,'핵가족':False,'대가족':False})
train_df['가족형태-핵가족'] = train_df['가족형태'].map({'싱글':False,'핵가족':True,'대가족':False})
train_df['가족형태-대가족'] = train_df['가족형태'].map({'싱글':False,'핵가족':False,'대가족':True})
train_df[['가족형태', '가족형태-싱글','가족형태-핵가족','가족형태-대가족']].head(15)

,가족형태,가족형태-싱글,가족형태-핵가족,가족형태-대가족
PassengerId,,,,
1,핵가족,False,True,False
2,핵가족,False,True,False
3,싱글,True,False,False
4,핵가족,False,True,False
5,싱글,True,False,False
6,싱글,True,False,False
7,싱글,True,False,False
8,대가족,False,False,True
9,핵가족,False,True,False


# 문제 17

`Name` 컬럼은 `성, 호칭. 이름` 구조를 가지고 있습니다. `호칭`을 추출해서, `호칭` 컬럼에 저장해주세요.

그리고, `호칭`별 생존율을 구해주세요.

In [23]:
def apply_fn(name):
    return name.split(".")[0].split(",")[1].strip()

train_df['호칭'] = train_df['Name'].apply(apply_fn)
#train_df[['Name','호칭']].head()

round(train_df.pivot_table(index='호칭', values='Survived'),2)

,Survived
호칭,
Capt,0.00
Col,0.50
Don,0.00
Dr,0.43
Jonkheer,0.00
Lady,1.00
Major,0.50
Master,0.57
Miss,0.70


# 문제 18

낮은 모수의 `호칭`들을 정리해봅시다.

`Mr`, `Mrs`, `Miss`, `Master`가 거의 대부분입니다. 이 외의 `호칭`은 `Other`로 정리해봅시다.

그리고, `호칭`별 생존율을 다시 구해봅시다.

In [24]:
conditions1 = [
    (train_df['호칭'] == 'Mr'),
    (train_df['호칭'] == 'Mrs'),
    (train_df['호칭'] == 'Miss'),
    (train_df['호칭'] == 'Master')]
choices1 = ['Mr', 'Mrs', 'Miss', 'Master']
train_df['호칭'] = np.select(conditions1, choices1, default='Other')

#train_df.loc[train_df['호칭']=='Other',['Name','호칭']]

round(train_df.pivot_table(index='호칭', values='Survived'),2)

,Survived
호칭,
Master,0.57
Miss,0.70
Mr,0.16
Mrs,0.79
Other,0.44


# 문제 19

`결혼유무` 컬럼을 만들어봅시다.

+ `호칭`이 `Mr`, `Mrs` 이면 : `True`
+ `호칭`이 `Master`, `Miss` 이면 : `False`
+ 나머지 `호칭`은 `NaN` 그대로 둡시다.

`결혼유무`별 생존율을 구해봅시다.

In [25]:
conditions2 = [
    (train_df['호칭'] == 'Mr') | (train_df['호칭'] == 'Mrs'),
    (train_df['호칭'] == 'Miss') | (train_df['호칭'] == 'Master')]
choices2 = ['True', 'False']
train_df['결혼유무'] = np.select(conditions2, choices2, default='NaN')

round(train_df.pivot_table(index='결혼유무', values='Survived'),2)

,Survived
결혼유무,
False,0.68
NaN,0.44
True,0.28
